<a href="https://colab.research.google.com/github/ahmedlabib02/Nlp-project/blob/main/Nlp_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Milestone 1

In [ ]:
import os 
import json
import re
import pandas as pd

### Data preparation

#### Unzipping the "B Hodoo2" channel folder

In [1]:
import zipfile
zip_path = "./B Hodoo2-20250226T182935Z-001.zip"
extract_path = "./"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


#### Unzipping the "Kefaya ba2a" channel folder

In [2]:
import zipfile
zip_path = "./Kefaya Ba2a-20250226T183527Z-001.zip"
extract_path = "./"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

### Building dict for annotations and text

In [53]:
def canonicalize(title):
    
    title = title.replace("_", " ")
   
    title = title.lower()
    
    title = re.sub(r'[^\w\s]', '', title)
    
    # title = re.sub(r'\bبودكاست بهدوء مع كريم\b', '', title)
    # title = re.sub(r'\bجلسة\b', '', title)
    
    title = re.sub(r'\s+', ' ', title).strip()
    return title

In [54]:
def get_category_for_filename(target_filename, annotations_path):
    """
    Load annotations from annotations_path and try to match the
    canonicalized target_filename (without file extension) to the canonicalized
    title from each annotation. If a match is found, return the annotation's category.
    """
    with open(annotations_path, 'r', encoding='utf-8') as f_json:
        annotations = json.load(f_json)
    
    base_target = os.path.splitext(target_filename)[0]
    target_can = canonicalize(base_target)
    for entry in annotations:
        annotation_title = entry.get('title')
        if annotation_title:
            
            annotation_can = canonicalize(annotation_title)
            if annotation_can == target_can:
                # print("Matched annotation:", entry.get('title'))
                return entry.get('category')
    return None

In [ ]:
def add_transcripts_from_folder(transcripts_dict, folder_path, channel):
    
    annotations_json = os.path.join(folder_path, 'annotations.json')
    raw_folder = os.path.join(folder_path, 'raw')
    for filename in os.listdir(raw_folder):
        full_path = os.path.join(raw_folder, filename)
        if os.path.isfile(full_path):
            with open(full_path, 'r', encoding='utf-8') as file:
                transcript = file.read()
                category = get_category_for_filename(filename, annotations_json)
            base_filename, _ = os.path.splitext(filename)
            transcripts_dict[base_filename] = (channel, category, transcript)
    return transcripts_dict

In [ ]:
transcripts_dict = {}
first_folder_path = './B Hodoo2/'
second_folder_path = './Kefaya Ba2a/'

# transcripts_dict = add_transcripts_from_folder(transcripts_dict, first_folder_path,  'B Hodoo2')
transcripts_dict = add_transcripts_from_folder(transcripts_dict, second_folder_path, 'Kefaya Ba2a')



{'اتقلوا تاخدوا حاجة نضيفة!!!!!': ('Kefaya Ba2a', 'People & Blogs', '[موسيقى]\nانت فكرنا فين\nالسيز احنا محتاجين الكفايه ب بجد ايوه\nوالله بجد ها يا لولو س 8\nامتى\nيابيش طب دلوقتي انا على رؤيتي\nالمجتمعيه ما كفايه ما كفايه والله العظيم\nكل بوست كل صوره كل فيديو بنزله فين الس\nجنيه فين الس جنيه تاخرت ليه مش هو انا مش\nانسان وبعدين في حاجه اسمها اتقل تاخد\nحاجه نظيفه صح ولا لا طيب تعال اوريك حاجه\nاهو ست جديد اهو ست ولا مش ست ده ها بص\nصور مطبوعه دي بفلوس ولا مش فلوس ها كفايه\nبقى بودجي محمد زبده كل ده مهديك كل دي\nحاجات طب تعالى شوف لي بقى انج الزاويه\nالتانيه بصلي كده اه هوب باسكت اهو ف في\nديكور اتعمل في حاجه اتعملت عشان نعمل\nحاجه نضيفه طب تعالالي اضاءه حمراء دي ولا\nلا دي اضاءه حمرا ولا لا زاويه تانيه ا\nكاميرا تانيه شايف اهو زاويه تانيه اهي\nوالناس قاعده ورا الكاميرا اهو الناس\nشغاله اهي ناس تعبانه اذا احنا بنحضر حاجه\nنظيفه بنحضر حاجه حلوه عشان اللي بعد كده\nيبقى اجمد فلو لقيت حد تاني جاي يقول لي\nفينس الجديد\nهطلع كفايه بقى كفايه بقى كفايه بقى كفايه\nبقى كفايه بقى كفايه بقى كفاي

In [ ]:
data = [(title, info[0], info[1], info[2]) for title, info in transcripts_dict.items()]
df = pd.DataFrame(data, columns=['title', 'channel', 'category', 'transcript'])
print(df.head())

                               title      channel        category  \
0      اتقلوا تاخدوا حاجة نضيفة!!!!!  Kefaya Ba2a  People & Blogs   
1   بودكاست كفاية بقى - الحصة الأولى  Kefaya Ba2a  People & Blogs   
2  بودكاست كفاية بقى - العشق الممنوع  Kefaya Ba2a  People & Blogs   
3   بودكاست كفاية بقى - العقل السليم  Kefaya Ba2a          Comedy   
4      بودكاست كفاية بقى - بس هو طيب  Kefaya Ba2a  People & Blogs   

                                          transcript  
0  [موسيقى]\nانت فكرنا فين\nالسيز احنا محتاجين ال...  
1  اهلا بكم مره اخرى في\nبودكاست المفضل كفايه بقى...  
2  [موسيقى]\nاهلا بكم في بودكاست كفايه بقى البودك...  
3  [موسيقى]\nاهلا بكم في بودكاست كفايه بقى البودك...  
4  اهلا بكم في بودكاست كفايه بقى البودكاست\nده يا...  
